In [17]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time
from random import randint
import PyCapture2 as pc2
import detect

%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
DAQ = daq.Daq(print_normal=True)

In [19]:
instr_oscope = detect.pyvisa()
instr_PS = detect.serial_ports()

In [39]:
print(instr_oscope)
print(instr_PS)

{'C046401': <instrInfo.TDS2024C object at 0x7fa3841c3e80>}
{'/dev/ttyUSB0': <instrInfo.FS304 object at 0x7fa3841e14e0>, 'NT2009101400': <instrInfo.KA3005P object at 0x7fa3841e1cc0>, '5573631333835150F150': <instrInfo.TC object at 0x7fa3841e14a8>, '55736313338351603181': <instrInfo.FRG700 object at 0x7fa3841e1da0>}


In [20]:
DAQ.connect_instr(instr_oscope['C046401'])
DAQ.connect_instr(instr_PS['NT2009101400'])

Power supply ID: KORADKA3005PV2.0
Instrument NT2009101400 connected and process started.
Oscilloscope ID: TEKTRONIX,TDS 2024C,C046401,CF:91.1CT FV:v24.26

Instrument C046401 connected and process started.
Instrument NT2009101400 disconnected.
Instrument C046401 disconnected.


In [21]:
DAQ.send_command(instr_PS['NT2009101400'], 'turn_on')
DAQ.send_command(instr_oscope['C046401'], 'acquire_off')
DAQ.send_command(instr_oscope['C046401'], 'set_acquisition_stop', "SEQuence")
DAQ.send_command(instr_oscope['C046401'], 'set_acquisition_mode', "AVErage")

In [26]:
Volts = [0]#np.linspace(0.1, 20, 200)
print(Volts)

[0]


In [27]:
chan = {'CH2' : [False, 'Voltage Drop'],
        'MATH' : [False, 'Biasing Voltage']}

for V in Volts:
    DAQ.send_command(instr_PS['NT2009101400'], 'set_voltage', V)
    print(V)
    for ch in chan:
        print('   '+ch)
        chan[ch][0] = True
        
        DAQ.send_command(instr_oscope['C046401'], 'set_data_source', ch)
        
        if ch=='MATH':
            DAQ.send_command(instr_oscope['C046401'], 'set_average_num', 4)
            delay = 2
        elif ch=='CH2':
            DAQ.send_command(instr_oscope['C046401'], 'set_average_num', 32)
            delay = 5
        
        time.sleep(.25)
        DAQ.send_command(instr_oscope['C046401'], 'acquire_on')
        
        time.sleep(delay)
        DAQ.send_command(instr_oscope['C046401'], 'save_waveform') 
        
        time.sleep(2)

        chan[ch][0] = False
        

0
   CH2
   MATH


In [28]:
DAQ.close_daq()

Checking that all threads exited.
Thread name: MainThread Alive: True
Checking that all threads exited.
Thread name: Thread-2 Alive: True
Checking that all threads exited.
Thread name: Thread-3 Alive: True
Checking that all threads exited.
Thread name: IPythonHistorySavingThread Alive: True
Checking that all threads exited.
Thread name: Thread-1 Alive: True
Checking that all threads exited.
Thread name: QueueFeederThread Alive: True
Checking that all threads exited.
Thread name: QueueFeederThread Alive: True
Checking that all threads exited.
Thread name: QueueFeederThread Alive: True
Checking that all threads exited.
Thread name: QueueFeederThread Alive: True
